# - Data Warehouse - Inserted


### Het samenvoegen van bestaande tabellen voor het uiteindelijke 'Datawarehouse'

##### Importeren van benodigde dependencies

In [1]:
import pandas as pd
import pyodbc

import warnings
warnings.filterwarnings("ignore")

# Importeren van de create_connection en run_query functies uit de database_utils.py file
from utils.database_utils import create_connection, run_query

##### 1 - Opbouwen van dataframes voor elke database-tabel

In [2]:
aw_BusinessEntityAddressDF = run_query("SELECT * FROM Person.BusinessEntityAddress", "AdventureWorks2019")
aw_AddressDF = run_query("SELECT * FROM Person.Address", "AdventureWorks2019")
aw_AddressTypeDF = run_query("SELECT * FROM Person.AddressType", "AdventureWorks2019")
aw_StateProvinceDF = run_query("SELECT * FROM Person.StateProvince", "AdventureWorks2019")
aw_CountryRegionDF = run_query("SELECT * FROM Person.CountryRegion", "AdventureWorks2019")

print("BusinessEntityAddressDF columns:", aw_BusinessEntityAddressDF.columns)
print("AddressDF columns:", aw_AddressDF.columns)
print("AddressTypeDF columns:", aw_AddressTypeDF.columns)
print("StateProvinceDF columns:", aw_StateProvinceDF.columns)
print("CountryRegionDF columns:", aw_CountryRegionDF.columns)



BusinessEntityAddressDF columns: Index(['BusinessEntityID', 'AddressID', 'AddressTypeID', 'rowguid',
       'ModifiedDate'],
      dtype='object')
AddressDF columns: Index(['AddressID', 'AddressLine1', 'AddressLine2', 'City', 'StateProvinceID',
       'PostalCode', 'SpatialLocation', 'rowguid', 'ModifiedDate'],
      dtype='object')
AddressTypeDF columns: Index(['AddressTypeID', 'Name', 'rowguid', 'ModifiedDate'], dtype='object')
StateProvinceDF columns: Index(['StateProvinceID', 'StateProvinceCode', 'CountryRegionCode',
       'IsOnlyStateProvinceFlag', 'Name', 'TerritoryID', 'rowguid',
       'ModifiedDate'],
      dtype='object')
CountryRegionDF columns: Index(['CountryRegionCode', 'Name', 'ModifiedDate'], dtype='object')


Je kunt vergelijkbare queries uitvoeren voor andere tabellen die je wilt opnemen.

#### 2 - Data transformatie

Na het opbouwen van de dataframes voor elke database-tabel, kunnen we beginnen met het transformeren van de data. Dit omvat het samenvoegen van tabellen, het toepassen van filters, het uitvoeren van berekeningen, enzovoort.

In [3]:
# Drop the 'rowguid' and 'ModifiedDate' columns from each DataFrame
aw_BusinessEntityAddressDF.drop(columns=['rowguid', 'ModifiedDate'], inplace=True)
aw_AddressDF.drop(columns=['rowguid', 'ModifiedDate'], inplace=True)
aw_AddressTypeDF.drop(columns=['rowguid', 'ModifiedDate'], inplace=True)
aw_StateProvinceDF.drop(columns=['rowguid', 'ModifiedDate'], inplace=True)
aw_CountryRegionDF.drop(columns=['ModifiedDate'], inplace=True)

# Merge the DataFrames
merged_df = pd.merge(aw_BusinessEntityAddressDF, aw_AddressDF, on='AddressID', how='inner')
merged_df = pd.merge(merged_df, aw_AddressTypeDF, on='AddressTypeID', how='inner')
merged_df = pd.merge(merged_df, aw_StateProvinceDF, on='StateProvinceID', how='inner')
merged_df = pd.merge(merged_df, aw_CountryRegionDF, on='CountryRegionCode', how='inner')

filtered_df = merged_df.rename(columns={'Name': 'Country', 'Name_x': 'AddressType', 'Name_y': 'State'})
filtered_df = filtered_df.drop_duplicates(subset='BusinessEntityID', keep='first')
filtered_df

,BusinessEntityID,AddressID,AddressTypeID,AddressLine1,AddressLine2,City,StateProvinceID,PostalCode,SpatialLocation,AddressType,StateProvinceCode,CountryRegionCode,IsOnlyStateProvinceFlag,State,TerritoryID,Country
0,1,249,2,4350 Minute Dr.,None,Newport Hills,79,98006,b'\xe6\x10\x00\x00\x01\x0c\x0b]{e\xaf\xccG@\x0...,Home,WA,US,False,Washington,1,United States
1,2,293,2,7559 Worth Ct.,None,Renton,79,98055,"b'\xe6\x10\x00\x00\x01\x0c\x87""s}.\xc2G@7\x86\...",Home,WA,US,False,Washington,1,United States
2,3,224,2,2137 Birchwood Dr,None,Redmond,79,98052,b'\xe6\x10\x00\x00\x01\x0c\xeb\x81f\x0e\xd7\xd...,Home,WA,US,False,Washington,1,United States
3,4,11387,2,5678 Lakeview Blvd.,None,Minneapolis,36,55402,b'\xe6\x10\x00\x00\x01\x0c\xf6\xda\x1c\x0f\x1a...,Home,MN,US,False,Minnesota,3,United States
4,5,190,2,9435 Breck Court,None,Bellevue,79,98004,b'\xe6\x10\x00\x00\x01\x0c-\x18\x1e;H\xc9G@\xe...,Home,WA,US,False,Washington,1,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19609,20773,29879,2,988 Mt. Everest Court,None,W. York,14,BD1 4SJ,b'\xe6\x10\x00\x00\x01\x0c8\xc5\x979\x14\xefJ@...,Home,ENG,GB,True,England,10,United Kingdom
19610,20774,29880,2,7413 Alpine Drive,None,Torrance,9,90505,b'\xe6\x10\x00\x00\x01\x0c\xce\xd4\xd2\\\xd4\x...,Home,CA,US,False,California,4,United States
19611,20775,29881,2,4764 East Avenue,None,Bremerton,79,98312,b'\xe6\x10\x00\x00\x01\x0c5\xc4\xc9\x83\x86\xc...,Home,WA,US,False,Washington,1,United States
19612,20776,29882,2,"34334, rue Jean Mermoz",None,Versailles,164,78000,b'\xe6\x10\x00\x00\x01\x0c\xd9$\x1f\x00\x1fdH@...,Home,78,FR,False,Yveline,7,France


In [ ]:
create_table_query = """

CREATE TABLE AddressInfo (
    BusinessEntityID int PRIMARY KEY,
    AddressID int,
    AddressTypeID int,
    AddressLine1 nvarchar(max),
    AddressLine2 nvarchar(max),
    City nvarchar(max),
    StateProvinceID int,
    PostalCode nvarchar(max),
    SpatialLocation geography,
    AddressType nvarchar(max),
    StateProvinceCode nchar(10),
    CountryRegionCode nvarchar(max),
    IsOnlyStateProvinceFlag bit,
    State nvarchar(max),
    TerritoryID int,
    Country nvarchar(max),
);
"""

# Create the table in SQL Server
# db_name = "testDB"
conn, cursor = create_connection()
cursor.execute(create_table_query)
conn.commit()
conn.close()

#### 3 - Data loading

Na het transformeren van de data, kunnen we de resulterende dataframe in de doeltabel van ons datawarehouse laden.

In [ ]:
test_database_name = 'testDB'
conn, cursor = create_connection(test_database_name)

for index, row in filtered_df.iterrows():
    # Convert binary data to hexadecimal
    spatial_location = row['SpatialLocation'].hex()

    # Convert Python boolean to SQL boolean
    is_only_state_province_flag = 1 if row['IsOnlyStateProvinceFlag'] else 0

    # Escape single quotes in string data
    address_line1 = row['AddressLine1'].replace("'", "''") if row['AddressLine1'] else None
    address_line2 = row['AddressLine2'].replace("'", "''") if row['AddressLine2'] else None
    city = row['City'].replace("'", "''") if row['City'] else None
    address_type = row['AddressType'].replace("'", "''") if row['AddressType'] else None
    state_province_code = row['StateProvinceCode'].replace("'", "''") if row['StateProvinceCode'] else None
    country_region_code = row['CountryRegionCode'].replace("'", "''") if row['CountryRegionCode'] else None
    state = row['State'].replace("'", "''") if row['State'] else None
    country = row['Country'].replace("'", "''") if row['Country'] else None

    query = f"""
        INSERT INTO AddressInfo  (
            BusinessEntityID, AddressID, AddressTypeID, AddressLine1, AddressLine2, City, 
            StateProvinceID, PostalCode, SpatialLocation, AddressType, 
            StateProvinceCode, CountryRegionCode, IsOnlyStateProvinceFlag, State, 
            TerritoryID, Country
        ) 
        VALUES (
            {row['BusinessEntityID']}, {row['AddressID']}, {row['AddressTypeID']}, '{address_line1}', 
            '{address_line2}', '{city}', {row['StateProvinceID']}, '{row['PostalCode']}', 
            0x{spatial_location}, '{address_type}', '{state_province_code}', 
            '{country_region_code}', {is_only_state_province_flag}, '{state}', 
            {row['TerritoryID']}, '{country}'
        )
    """
    print(query)  # Print the query
    cursor.execute(query)

conn.commit()
conn.close()

**Note:** Voeg indien nodig zoveel Markdown- of codeblokken toe als nodig is.

#### 4 -  Data Quality Checks

Voeg controles toe om de kwaliteit van de gegevens te waarborgen voordat ze worden geladen in het datawarehouse:

In [ ]:
# Controleren op ontbrekende waarden
missing_values = filtered_df.isnull().sum()

# Controleren op duplicaten
duplicate_rows = filtered_df.duplicated().sum()

# Weergave van resultaten
print("Aantal ontbrekende waarden:", missing_values)
print("Aantal duplicaten:", duplicate_rows)


**Note:** Dit is optioneel, het leek mij opzich best handig om te doen.